In [ ]:
def parse_easylist_selectors(file_path):
    selectors = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Only consider lines with CSS selector-based rules
            if line.startswith('##'):
                selector = line[2:].strip()
                selectors.append(selector)
    return selectors

parse_easylist_selectors("easylist.txt")

In [ ]:
import requests
from bs4 import BeautifulSoup

def check_ads(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Check for common ad-related elements or scripts
        ad_elements = soup.find_all(['iframe', 'script', 'div'], 
                                    id=lambda x: x and 'ad' in x.lower(),
                                    class_=lambda x: x and 'ad' in x.lower())

        return len(ad_elements) > 0
    except Exception as e:
        print(f"Error occurred: {e}")
        return False

# Example usage
url = 'https://www.espncricinfo.com/'
has_ads = check_ads(url)
print(f"Ads found: {has_ads}")


In [ ]:
import pandas as pd
import random


In [ ]:
df = pd.read_csv("/Users/kaleemullahqasim/Downloads/top-1m.csv")

In [ ]:
df

In [ ]:
filtered_df = df.iloc[150000:-150000]

# Randomly sample 1000 domains
random_domains = filtered_df['google.com'].sample(n=1000, random_state=random.seed()).tolist()


In [ ]:
random_domains

In [ ]:
formatted_domains = ['https://' + domain for domain in random_domains]

# Output the list
for domain in formatted_domains:
    print(domain)

In [14]:
domains_data = pd.DataFrame(formatted_domains)

In [15]:
domains_data.to_parquet('urls.parquet')

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import os

def download_ads(url, save_dir='./ads'):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Identify ad elements using CSS classes, attributes, or patterns
        ad_elements = soup.find_all(
            'div', class_=['my-ad', 'ad-container', 'adsense-unit']
        )  # Replace with appropriate selectors

        # Extract ad information and download images
        for ad_element in ad_elements:
            image_url = ad_element.find('img')['src']
            if image_url:
                # Download ad image
                filename = os.path.basename(image_url)
                save_path = os.path.join(save_dir, filename)

                try:
                    download_image(image_url, save_path)
                except Exception as e:
                    print(f"Error downloading image: {e}")
                    continue

def download_image(image_url, save_path, retry_attempts=3):
    for attempt in range(retry_attempts):
        try:
            image_response = requests.get(image_url)
            if image_response.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(image_response.content)
                break
        except Exception as e:
            print(f"Error downloading image: {e}")
            if attempt < retry_attempts - 1:
                print(f"Retrying attempt {attempt + 1}...")
                time.sleep(1)
            else:
                raise e

# Example usage
download_ads('https://www.espncricinfo.com/')
